In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyhive import presto
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")
import my_func

presto_conn = presto.connect(
    host='bi-presto.serving.data.production.internal',
    port=80,
    protocol='http',
    catalog='hive',
    username='mayank.jha@rapido.bike',
    # requests_kwargs=req_kw,
)

from pymongo import MongoClient
client16                            = MongoClient("mongodb://v-kubernetes-acl--prod-mongo-subs-vWmvA3LNGNAq56t9SsN0-1651058959:4bcy-bdMB9mmiKKKwq0Y@subscription-mongo-4.db.backend.production.internal:27107/rapidoPass?authSource=rapidoPass")
db16                                = client16['rapidoPass']
subscriptionCollection                = db16['captainSubscriptions']

In [2]:
# Cities

def get_cities():

    orders_query = """   
    select _id as city_id, displayname as city_name from legacy.cities where active=true
                    
               """
    
    #print(orders_query)
    df_orders_data = pd.read_sql(orders_query, presto_conn)
    return df_orders_data

df_city = get_cities()
#df_city.head()

In [4]:
df_subscription = pd.io.json.json_normalize(list(subscriptionCollection.find({})))
df_subscription = df_subscription[['userId','city','startTime','endTime','status','paymentType','amount','active','subscriptionRule.userSelectors']]
df_subscription = df_subscription[['userId','city','startTime','endTime','status','paymentType','amount']]
df_subscription['startTime'] = df_subscription['startTime'].apply(lambda x : (datetime.utcfromtimestamp(x)+timedelta(minutes = 330)).strftime('%Y-%m-%d'))
df_subscription['endTime'] = df_subscription['endTime'].apply(lambda x : (datetime.utcfromtimestamp(x)+timedelta(minutes = 330)).strftime('%Y-%m-%d'))


df_subscription = df_subscription.merge(df_city, how = 'left', left_on = 'city', right_on = 'city_id')
df_subscription['city'] = df_subscription['city_name']
df_subscription.drop(['city_id','city_name'], axis = 1, inplace = True)
df_subscription['week'] = df_subscription['endTime'].apply(lambda x : pd.to_datetime(x).strftime('%V')).astype(int)
df_subscription = df_subscription[(df_subscription['city']=='Kolkata') & (df_subscription['status']=='SUCCESS')]

In [6]:
df_subscription.drop_duplicates('userId', inplace=True)

In [9]:
df_subscription = df_subscription[['userId']].rename(columns={'userId':'captain_id'})

In [15]:
start_date = '2022-04-18'
end_date = '2022-04-24'
city = 'Kolkata'

In [17]:
# POC Segment

def get_poc(dt, city):
    print("fetching poc segments for : ",dt)
    orders_query = """   select cq.captain_id, 
                                csv.mobilenumber,
                                cq.city_name as city,
                                cq.segment as poc_segment,
                                substr(cq.recency_segment, 1, strpos(cq.recency_segment, '_')-1) as recency
                            from
                                datasets.captain_quality_segments cq
                                    left join datasets.captain_single_view csv
                                        on cq.captain_id = csv.captainid
                                where cq.yyyymmdd = '{sd}'
                                    and cq.city_name = '{ct}'
               """.format(sd = dt.replace('-',''), ct = city)
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched segment data : ", len(df_lh))

    return df_lh

poc_segment = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 6]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    segment = get_poc(day, city)
    segment['week'] = pd.to_datetime(day).strftime('%Y-%V')
    poc_segment = pd.concat([poc_segment,segment])

poc_segment.head()

fetching poc segments for :  2022-04-24
 fetched segment data :  22227


,captain_id,mobilenumber,city,poc_segment,recency,week
0,5d5cb984a24c96105e1c447c,9831457038,Kolkata,LP_Intra_MO,active,2022-16
1,6028b50a19e4982abe1e2680,8617243976,Kolkata,ZP,churn,2022-16
2,6143f628de864c0664ff2602,9874923139,Kolkata,MP_Intra_UHO,active,2022-16
3,622eba81bbd060ce1ce91d84,8910529658,Kolkata,LP_D_UHO,dormant,2022-16
4,611e503934b93d7e6e2c812f,9038942717,Kolkata,ZP,churn,2022-16


In [ ]:
df_50 = df_subscription[df_subscription['captain_id']]